In [1]:
import os

import pandas as pd

from IPython.display import IFrame

In [2]:
IFrame('https://dbdiagram.io/embed/5f876a5e3a78976d7b77adf4', width=800, height=500)

## Documentation
Additional inline schema documentation can be found by navigating to [DB Diagram](https://dbdiagram.io/d/5f876a5e3a78976d7b77adf4)

## Clean Data and Schema Definitions
Below are rudimentary representations of the various tables represented in the diagram above.

In [3]:
# List the files we have just in our data definition folder for just reference
# We could loop through these CSVs and read them in within the loop, but we'll manually inspect the DFs to 
# view the actual data
parent = 'db'
for fn in sorted(os.listdir(os.path.join('.', parent))):
    if fn.endswith('.csv'):
        print(fn)

accounts.csv
leads.csv
opps.csv
orders.csv
product.csv
users.csv


In [4]:
def fetch_df_from_file(filename, parent_dir):
    csv = filename.lower() + '.csv'
    return pd.read_csv(os.path.join(parent, csv))

In [5]:
ACCOUNTS = fetch_df_from_file('accounts', parent)
ACCOUNTS.head()

,billing_address_short,billing_city,billing_country,id,name
0,58343 Lisa Plaza,North Marcusmouth,Mauritania,1,Bradley and Sons
1,1942 Vaughn Terrace Apt. 365,Jasonmouth,Chile,2,Benitez Ltd
2,478 Isaiah Plains Suite 337,Pamelaside,Barbados,3,Perez Group
3,2553 Russell Road,New Justinborough,Timor-Leste,4,Wright-Waters
4,585 Palmer Ports Suite 058,Lake Shanechester,Greece,5,"Hines, Herman and Gibbs"


In [6]:
ACCOUNTS = ACCOUNTS[['id','name', 'billing_address_short', 'billing_city', 'billing_country']]

In [7]:
LEADS = fetch_df_from_file('leads', parent)
LEADS.head()

,account_id,id,name,owner_id
0,9,1,Brittany Shields,11
1,1,2,Robert Francis,12
2,3,3,Selena Becker,14
3,7,4,David Olson,6
4,1,5,Diana Potter,9


In [8]:
LEADS = LEADS[['id', 'name', 'account_id', 'owner_id']]

In [9]:
OPPS = fetch_df_from_file('opps', parent)
OPPS.head()

,account_id,date_closed,deal_length,deal_value,id,is_closed,is_lost,is_won,lead_id,mrr,name,owner_id,stage
0,4,2020-10-12,9,16690.62,1,True,True,NaN,46,1854.51,facilitate real-time schemas,1,closed_lost
1,2,NaN,3,19789.60,2,False,NaN,NaN,554,6596.53,syndicate innovative action-items,17,qualified
2,8,NaN,4,878.71,3,False,NaN,NaN,889,219.68,generate intuitive technologies,24,qualified
3,2,2020-05-21,6,19099.42,4,True,NaN,True,540,3183.24,aggregate user-centric functionalities,3,closed_won
4,6,2020-04-19,2,24778.21,5,True,True,NaN,863,12389.10,target plug-and-play models,16,closed_lost


In [10]:
keep_cols = ['id', 'name', 'owner_id', 'account_id', 'lead_id', 'stage', 'is_closed',
             'is_won', 'is_lost', 'mrr', 'deal_value', 'deal_length']
OPPS = OPPS[keep_cols]

In [11]:
ORDERS = fetch_df_from_file('orders', parent)
ORDERS.head()

,created_at,id,opportunity_id,products
0,2019-03-26,0,688,[]
1,2020-09-23,1,293,[]
2,2020-05-22,2,912,"[1, 2]"
3,2020-05-14,3,404,[1]
4,2020-05-02,4,381,"[1, 2]"


In [12]:
ORDERS = ORDERS[['id', 'opportunity_id', 'products', 'created_at']]

In [13]:
PRODUCT = fetch_df_from_file('product', parent)
PRODUCT.head()

,id,type
0,1,database
1,2,prof_consulting
2,3,self_monitor


In [14]:
USERS = pd.read_csv(os.path.join(parent, 'users.csv'))
USERS.head()

,date_inactive,date_joined,did_hit_quota,full_name,id,is_active,role,role_id,team,team_id
0,NaN,2019-04-05,False,Joseph Dougherty,1,True,"{'id': 2, 'monthly_quota': 2220.7615057236508,...",2,"{'id': 0, 'type': 'enterprise_sales', 'divisio...",0
1,2020-09-03,2019-10-26,False,Michelle Hunter,2,False,"{'id': 1, 'monthly_quota': 2145.339173943017, ...",1,"{'id': 0, 'type': 'enterprise_sales', 'divisio...",0
2,NaN,2020-02-24,True,Robert Frazier,3,True,"{'id': 1, 'monthly_quota': 32233.03942450081, ...",1,"{'id': 1, 'type': 'smb_sales', 'division': 'NA...",1
3,2020-02-28,2019-08-18,False,Elizabeth Cuevas MD,4,False,"{'id': 2, 'monthly_quota': 43630.63254267283, ...",2,"{'id': 1, 'type': 'smb_sales', 'division': 'EM...",1
4,NaN,2020-02-26,False,Charles Lee,5,True,"{'id': 1, 'monthly_quota': 43619.69858575248, ...",1,"{'id': 1, 'type': 'enterprise_sales', 'divisio...",1


In [15]:
user_columns = ['id', 'full_name', 'role_id', 'role', 'team', 'team_id',
                'did_hit_quota', 'date_joined', 'is_active', 'date_inactive']
USERS = USERS[user_columns]

---

## Questions of Interest
- Do our most productive (frequently hit quota) Sales Reps generate the most outbound phone calls / secure product demos? (Sales Activity)
- What are the top X churning accounts (accounts that dropped spend) over the last 90 days?
- What % of our assigned Sales Accounts in our EMEA region are spending at least $XM dollars? 
- How many Sales Reps are in the Enterprise Sales Segment? How many unique teams exist? 

---

Queries below use Snowflake syntax

#### Who are our most tenured Sales Reps / Leaders per Region?
```sql
SELECT
    U.full_name,
    R.title,
    T.division,
    DATE_DIFF('YEAR', U.date_joined, DATE_TRUNC('DAY', NOW())) / 365 AS tenure_in_yrs
FROM users U
JOIN role R 
    ON U.role_id = R.id
JOIN team T
    ON U.team_id = T.id
ORDER BY tenure_in_yrs DESC
```

#### What is our attrition rate on highly productive Sales Reps? (Productive meaning high revenue generating reps)
##### attrition rate in % = (Number of employees who leave ÷ number of employees) x 100

```sql

-- Get the top 20% of reps based on deal value
WITH top_reps AS (
    SELECT
        DISTINCT U.id,
        SUM(O.deal_value) AS deal_sum
    FROM users U
    JOIN role R
        ON U.role_id = R.id
    JOIN opps O
        ON O.owner_id = U.id
    WHERE R.title = 'sales_rep'
        AND O.is_won = True
    GROUP BY U.id
    HAVING PERCENT_RANK() OVER (ORDER BY deal_sum) > 0.8
),
-- Add an easy-to-aggregate flag to users 
flagged_users AS (
    SELECT
        DISTINCT U.id,
        CASE 
            WHEN U.is_active = False THEN 1 ELSE 0
        END AS did_leave
    FROM users U
)
-- Inner Join leaves only the Reps we care about
-- This naively does not take into account Year of departure
SELECT
    SUM(did_leave) / COUNT(DISTINCT T.id) * 100.0 AS attrition_rate
FROM top_reps T
JOIN users U 
    ON T.id = U.id
```

#### What % of our Sales Reps / Sales Leaders hit their quota (revenue target) in the past 5 quarters? 
```sql
WITH reached_quota AS (
  SELECT
    id,
    CASE
        WHEN did_hit_quota = TRUE THEN 1 ELSE 0
    END AS hit_quota_flag
  FROM users U
-- This is a naive date check that would be better suited using a historical record table of quotas hit
-- However, we can assume that sales people do not hit their quotas prior to joining the org
  WHERE U.did_hit_quota = True
      AND U.date_joined > DATEADD('MONTH', -15, NOW())
)
SELECT
    R.title,   
    SUM(Q.hit_quota_flag) / COUNT(DISTINCT U.id) * 100.0 AS percent_hit_quota
FROM users U
LEFT JOIN reached_quota Q
    ON Q.id = U.id
LEFT JOIN role R
    U.role_id = R.id
GROUP BY R.title
```

### How many of our accounts in the prior quarter are brand new customers? (Aka first time they are spending with us) How many are existing customers? 
```sql
WITH ranked_opps AS (
    SELECT
        DISTINCT O.id,
        A.id,
        O.date_closed,
        QUARTER(O.date_closed) AS deal_quarter,
        
        -- First Day and Last Day of Previous Quarter
        CASE
            WHEN deal_quarter > FIRST_DAY('DAY', DATEADD(QUARTER, -1, NOW()))
                 AND deal_quarter < LAST_DAY('DAY', DATEADD(QUARTER, -1, NOW())) THEN 1 
                ELSE 0
            END AS closed_last_quarter,
        ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY TIMESTAMP date_closed DESC) AS rank,
        CASE 
            WHEN rank = 1 THEN 1 ELSE 0 
        END AS is_new_account
    FROM accounts A
    JOIN opps O
        ON A.id = O.account_id
)
SELECT
    SUM(is_new_account) AS new_accounts_previous_quarter,
    COUNT(DISTINCT A.id) - new_accounts_previous_quarter AS existing_accounts
FROM ranked_opps
```

### How many Sales Reps are in the Enterprise Sales Segment? How many unique teams exist? 
```sql
-- How many Sales Reps are in the Enterprise Sales Segment?
SELECT
    COUNT(U.id) AS enterprise_reps,
    COUNT(DISTINCT T.id) AS distinct_teams
FROM users U
JOIN team T
    ON U.team_id = T.id
WHERE T.type = 'enterprise_sales'
    AND R.title = 'sales_rep'
```